In [1]:
import pandas as pd
import sys
import os
import glob
sys.path.append('../src')
import cargadata as cdat
import datetime
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')

import plotly.express as px
import plotly.graph_objects as go

In [2]:
contenido = os.listdir('../data/')
contenido

['1_am_ca',
 '2020_meteorologia_cm.zip',
 '2021_meteorologia_cm.zip',
 '2_cm_ca',
 'calidad_aire_estaciones.csv',
 'descripcion-fichero-open-data-meteorologico.pdf',
 'informacion_estaciones_red_calidad_aire__ayunta.csv',
 'informacion_estaciones_red_calidad_aire__ayunta.geo',
 'zonas_calidad_aire']

In [3]:
con_a2021 = os.listdir('../data/1_am_ca/2020')
con_a2021

['2020_01.csv',
 '2020_02.csv',
 '2020_03.csv',
 '2020_04.csv',
 '2020_05.csv',
 '2020_06.csv',
 '2020_07.csv',
 '2020_08.csv',
 '2020_09.csv',
 '2020_10.csv',
 '2020_11.csv',
 '2020_12.csv']

In [4]:
df = pd.read_csv('../data/1_am_ca/2020/2020_01.csv', sep = ';')

Exploramos el data set suelto

In [5]:
df.sample()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
3563,28,79,54,14,28079054_14_6,2020,1,18,2.97,V,...,34.5,V,37.48,V,38.47,V,34.73,V,49.77,V


Vamos a gestionara los datos marcados como no válidos. si son no validos los voy a igualar al anterior si válido

In [6]:
df[f'V01'].value_counts()

V    4678
N      15
Name: V01, dtype: int64

Pacece que cada columna suel tener alrederor de 20 valores así


Primero voy a convertir estos valores en Nan;

In [7]:
horas = ['01', '02','03','04','05', '06', '07', '08', '09', '10', '11', '12',
         '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']

In [8]:
columns_names = ['provincia', 'municipio', 'estacion', 'magnitud', 'punto_muestreo',
       'year', 'month', 'day', 'h01', 'v01', 'h02', 'v02', 'h03', 'v03', 'h04',
       'v04', 'h05', 'v05', 'h06', 'v06', 'h07', 'v07', 'h08', 'v08', 'h09',
       'v09', 'h10', 'v10', 'h11', 'v11', 'h12', 'v12', 'h13', 'v13', 'h14',
       'v14', 'h15', 'v15', 'h16', 'v16', 'h17', 'v17', 'h18', 'v18', 'h19',
       'v19', 'h20', 'v20', 'h21', 'v21', 'h22', 'v22', 'h23', 'v23', 'h24',
       'v24']

In [9]:
df.set_axis(columns_names, axis='columns', inplace=True)

In [10]:
df.sample()
    

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,h01,v01,...,h20,v20,h21,v21,h22,v22,h23,v23,h24,v24
1730,28,79,24,43,28079024_43_2,2020,1,9,1.31,V,...,1.18,V,1.14,V,1.13,V,1.16,V,1.15,V


Aplico la magia de Ester😸🐈🐅🐆🐅 a todas mis columnas

In [11]:
for hora in horas:
    df[f'h{hora}'] = np.where(df[f'v{hora}']=='N', np.nan, df[f'h{hora}'])

Compruebo que efectivamente se ha puesto los nan donde corresponde.

In [12]:
for hora in horas:
    print(df[f'v{hora}'].value_counts())
    print(df[f'h{hora}'].isna().sum())

V    4678
N      15
Name: v01, dtype: int64
15
V    4676
N      17
Name: v02, dtype: int64
17
V    4671
N      22
Name: v03, dtype: int64
22
V    4675
N      18
Name: v04, dtype: int64
18
V    4672
N      21
Name: v05, dtype: int64
21
V    4663
N      30
Name: v06, dtype: int64
30
V    4676
N      17
Name: v07, dtype: int64
17
V    4673
N      20
Name: v08, dtype: int64
20
V    4646
N      47
Name: v09, dtype: int64
47
V    4607
N      86
Name: v10, dtype: int64
86
V    4615
N      78
Name: v11, dtype: int64
78
V    4602
N      91
Name: v12, dtype: int64
91
V    4582
N     111
Name: v13, dtype: int64
111
V    4589
N     104
Name: v14, dtype: int64
104
V    4624
N      69
Name: v15, dtype: int64
69
V    4649
N      44
Name: v16, dtype: int64
44
V    4622
N      71
Name: v17, dtype: int64
71
V    4627
N      66
Name: v18, dtype: int64
66
V    4655
N      38
Name: v19, dtype: int64
38
V    4654
N      39
Name: v20, dtype: int64
39
V    4665
N      28
Name: v21, dtype: int64
28
V    4675
N

In [14]:
df.sample()

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,h01,v01,...,h20,v20,h21,v21,h22,v22,h23,v23,h24,v24
2855,28,79,40,12,28079040_12_8,2020,1,23,28.0,V,...,90.0,V,93.0,V,66.0,V,41.0,V,33.0,V


Dropeo las columnas de verificacion

In [15]:
li_v = []
for hora in horas:
    li_v.append(f'v{hora}')    

In [16]:
df.drop(li_v, axis = 1)

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,h01,h02,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
0,28,79,4,1,28079004_1_38,2020,1,1,7.00,8.00,...,13.00,12.00,11.00,10.00,10.00,12.00,14.00,12.00,11.00,9.00
1,28,79,4,1,28079004_1_38,2020,1,2,8.00,8.00,...,14.00,12.00,11.00,10.00,11.00,14.00,14.00,15.00,12.00,10.00
2,28,79,4,1,28079004_1_38,2020,1,3,9.00,8.00,...,9.00,10.00,11.00,12.00,12.00,12.00,13.00,13.00,10.00,8.00
3,28,79,4,1,28079004_1_38,2020,1,4,7.00,5.00,...,9.00,9.00,9.00,9.00,10.00,10.00,11.00,10.00,8.00,9.00
4,28,79,4,1,28079004_1_38,2020,1,5,8.00,6.00,...,14.00,11.00,11.00,11.00,11.00,12.00,14.00,13.00,11.00,9.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4688,28,79,60,14,28079060_14_6,2020,1,27,30.47,28.57,...,12.64,15.43,35.52,55.73,45.52,53.66,55.10,54.08,62.30,65.94
4689,28,79,60,14,28079060_14_6,2020,1,28,65.97,67.20,...,58.82,59.50,58.85,54.94,51.97,45.63,45.63,49.11,54.61,58.76
4690,28,79,60,14,28079060_14_6,2020,1,29,61.76,59.95,...,60.06,58.56,58.62,54.71,52.95,51.45,46.65,51.23,53.76,57.38
4691,28,79,60,14,28079060_14_6,2020,1,30,58.83,60.27,...,60.87,65.56,68.03,59.34,54.16,48.83,49.70,55.40,63.69,66.70


In [ ]:
df['h01'] = np.where(df['v01']=='N', np.nan, df['h01'])

In [ ]:
df[f'h01'].isna().sum()

In [ ]:
df.loc[df[f'v01']=='N',df[f'h01']]='NaN'